# Estimating the Test Error

In the previous notebook, we learned about _training error_, which is the error calculated on the training data. Training error is easy to calculate because the true labels are available in the training data. However, training error is not always a good measure of a model's quality, since a model that _overfits_ to the training data may have artificially low training error.

Ideally, we would like to evaluate regression models based on their _test error_, which is the error calculated on the test data. The problem with test error is that it is usually impossible to calculate, since the true labels are rarely available in the test data. In this section, we discuss strategies for estimating the test error using only the training data.

We'll use the Bordeaux wine data as example. Recall that our target is wine quality, which we measure as the log of price. The labels are the values of log(price), which we have observed for years up to 1980. That is, the observations up to 1980 comprise the training data. The observations after 1980&mdash;for which log(price) is unknown&mdash;is the test data.

In [1]:
import pandas as pd
import numpy as np

# Extract the training data.
data_dir = "https://dlsun.github.io/pods/data/"
bordeaux_df = pd.read_csv(data_dir + "bordeaux.csv",
                          index_col="year")

bordeaux_train = bordeaux_df.loc[:1980].copy()
bordeaux_train["log(price)"] = np.log(bordeaux_train["price"])

## Validation Error

To estimate the test error, we split the training data into a _training set_ and a _validation set_. First, the model is fit to just the data in the training set. Then, the model is evaluated based on its predictions on the validation set. Because the model did not train on any of the labels in the validation set, the validation set essentially plays the role of the test data, even though it was carved out of the training data.

The prediction error on the validation set is known as the _validation error_. The validation error is an approximation to the test error.

To split our data into training and validation sets, we can use the `.sample()` function in `pandas`. Let's use this to split our data into two equal halves, which we will call `train` and `val`.

In [2]:
train = bordeaux_train.sample(frac=0.5)
val = bordeaux_train.drop(train.index)

Here is the training set.

In [3]:
train

,price,summer,har,sep,win,age,log(price)
year,,,,,,,
1966,47.0,16.5,86,18.4,819,26,3.850148
1964,31.0,17.3,96,18.8,402,28,3.433987
1965,11.0,15.4,267,14.8,602,27,2.397895
1961,100.0,17.3,38,20.4,830,31,4.605170
1976,25.0,17.6,247,16.1,418,16,3.218876
1980,14.0,16.0,74,18.4,578,12,2.639057
1978,27.0,15.8,51,17.4,763,14,3.295837
1957,22.0,16.1,110,16.2,420,35,3.091042
1959,66.0,17.5,187,18.7,485,33,4.189655


Here is the validation set

In [4]:
val

,price,summer,har,sep,win,age,log(price)
year,,,,,,,
1952,37.0,17.1,160,14.3,600,40,3.610918
1955,45.0,17.1,130,16.8,502,37,3.806662
1958,18.0,16.4,187,19.1,582,34,2.890372
1960,14.0,16.4,290,15.8,763,32,2.639057
1962,33.0,16.3,52,17.2,697,30,3.496508
1963,17.0,15.7,155,16.2,608,29,2.833213
1967,19.0,16.2,118,16.5,714,25,2.944439
1969,12.0,16.5,244,16.6,575,23,2.484907
1970,40.0,16.7,89,18.0,622,22,3.688879


Now let's use these training/validation sets to approximate the test MSE of the 5-nearest neighbors model, based on **win** and **summer**, that we have fit previously.

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline

# extract features and label from training set
X_train = train[["win", "summer"]]
y_train = train["log(price)"]

# define pipeline and fit to training set
pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=5)
)
pipeline.fit(X=X_train, y=y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsregressor', KNeighborsRegressor())])

We make predictions on the validation set and calculate the validation RMSE:

In [6]:
from sklearn.metrics import mean_squared_error

# extract features and label from validation set
X_val = val[["win", "summer"]]
y_val = val["log(price)"]

# get model's predictions on validation set
y_val_ = pipeline.predict(X_val)

# calculate RMSE on validation set
rmse = np.sqrt(mean_squared_error(y_val, y_val_))
rmse

0.4650790869716535

Notice that the test error is higher than the training error that we calculated in the previous lesson. In general, this will be true. It is harder for a model to predict for new observations it has not seen, than for observations it has seen!

## Cross Validation

The validation error above was calculated using only 50% of the training data, since we split the training data in half to create the validation set. As a result, the estimate is noisy.

There is a cheap way to obtain a second opinion about how well our model will do on future data. Previously, we split our data at random into two halves, fitting the model to the first half and evaluating it on the second half. Because the model has not already seen the second half of the data, this approximates how well the model would perform on future data.

But the way we split our data was arbitrary. We might as well swap the roles of the two halves, fitting the model to the _second_ half and evaluating it on the _first_ half. As long as the model is always evaluated on data that is different from the data that was used to train it, we have a valid estimate of test error. A schematic of this approach, known as **cross-validation**, is shown below.

![](https://github.com/dlsun/pods/blob/master/05-Regression-Models/cross-validation.png?raw=1)

Because we will be doing all computations twice, just with different data, let's wrap the $k$-nearest neighbors algorithm above into a function called `get_val_error()`, that computes the validation error given training and validation data.

In [7]:
def get_val_error(train, val):

    # extract features and label from training set.
    X_train = train[["win", "summer"]]
    y_train = train["log(price)"]

    # define pipeline and fit to training set
    pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=5)
    )
    pipeline.fit(X=X_train, y=y_train)

    # extract features and label from validation set
    X_val = val[["win", "summer"]]
    y_val = val["log(price)"]

    # get model's predictions on validation set
    y_val_ = pipeline.predict(X_val)

    # calculate RMSE on validation set
    rmse = np.sqrt(mean_squared_error(y_val, y_val_))

    return rmse

If we apply this function to the training and validation sets from earlier, we get the same estimate of the test error.

In [8]:
get_val_error(train, val)

0.4650790869716535

But if we reverse the roles of the training and validation sets, we get another estimate of the test error.

In [9]:
get_val_error(val, train)

0.6167043190959203

Now we have two, somewhat independent estimates of the test error, which could be quite different. It is common to average the two numbers to obtain an overall estimate of the test error, called the **cross-validation estimate of test error**. Notice that the cross-validation estimate uses each observation in the data exactly once. We make a prediction for each observation, but always using a model that was trained on data that does not include that observation.

In [10]:
(get_val_error(train, val) + get_val_error(val, train)) / 2

0.5408917030337869

## Cross-Validation in scikit-learn

As you know by now, scikit-learn provides functions that automate routine tasks of machine learning. For cross-validation, there is a function, `cross_val_score`, that takes in a model (or pipeline), the training data, and a scoring function, and carries out all aspects of cross-validation, including

1. splitting the training data into training and validation sets
2. fitting the model to each training set
3. calculating the model's predictions on the corresponding validation set
4. calculating the score of the predictions on each validation set.

In [11]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline,
                         X=bordeaux_train[["win", "summer"]],
                         y=bordeaux_train["log(price)"],
                         scoring="neg_mean_squared_error",
                         cv=2)
scores

array([-0.47767636, -0.38655855])

First, notice that there are 2 scores. This is because scikit-learn calculated a score from each half of the data when that half served as the validation set.

Second, observe that the scores are negative. This is because scikit-learn requires that a "score" be something that ought to be maximized. Since we want to minimize the mean-squared error, we want to maximize the *negative* mean-squared error. Therefore, the scores that are reported here are the negative of the MSE.

To calculate the RMSE, we negate the negative sign and take a square root.

In [12]:
np.sqrt(-scores)

array([0.69114134, 0.62173833])

The average of these two scores is the cross-validation estimate of test error.

In [13]:
np.sqrt(-scores).mean()

0.6564398373317621

Note that this is not the same estimate of test error we got from scratch. The difference is due to the random splitting of the data into training and validation sets.

## $K$-Fold Cross Validation

One problem with splitting the training data into two halves is that the model is now trained on only half the amount of data. This model will likely perform worse than the actual model, which is trained on all of the training data. So the cross-validation estimate of test error is unnecessarily pessimistic.

We would like the size of the training set to be closer to the size of the original training data. We can do this by splitting the data into more than two subsamples. In general, we can split the data into $K$ subsamples, alternately training the data on $K-1$ subsamples and evaluating the model on the $1$ remaining subsample, i.e., the validation set. This way, we use $100(1 − 1/K)$ percent of the data for training in each of the "folds". This produces $K$ somewhat independent estimates of the test error. This procedure is known as **$K$-fold cross validation**. (Be careful not to confuse this $K$ with the $k$ in $k$-nearest neighbors.) In hindsight, the  cross validation that we were doing previously is $2$-fold cross validation.

A schematic of $4$-fold cross validation is shown below.

![](https://github.com/dlsun/pods/blob/master/05-Regression-Models/k-folds.png?raw=1)

Implementing $K$-fold cross validation in scikit-learn is easy. We simply set the `cv=` parameter to the desired number of folds. For example, the following code carries out $4$-fold cross validation.

In [14]:
scores = cross_val_score(pipeline,
                         X=bordeaux_train[["win", "summer"]],
                         y=bordeaux_train["log(price)"],
                         scoring="neg_mean_squared_error",
                         cv=4)
scores

array([-0.47213584, -0.30601689, -0.36874833, -0.11290777])

Notice that $K$ scores are produced&mdash;4 in this case&mdash;one from each fold. These scores can be averaged to produce a single estimate of the test error.

In [15]:
np.sqrt(-scores).mean()

0.545893343925109

How many folds to use in cross validation? In practice, 5-fold and 10-fold cross-validation are commonly used, and this is primarily what we'll do.

Another common choice is to set $K$ equal to $n$, the number of observations in the training data, resulting in **leave one out (LOO)** cross validation.

When choosing the number of folds there is a bias-variability tradeoff. Fewer folds introduces more bias, since in each fold only a subset of the data is used to fit the model (e.g., only half the data when $K=2$), and models fit on less data tend to perform less well and tend to overestimate the test error rate. With many folds&mdash;such as LOO at the extreme&mdash;there is less bias in estimating the test error because the training set within each fold is roughly the same size as the training data.


However, more folds leads to more variability in estimates of test error. With many folds, the training set is roughly the same across the folds resulting in estimates of test error that are highly positively correlated. With fewer folds, there is less overlap between training sets across the folds, therefore resulting in $K$ roughly independent measurements of test error. The mean of positively correlated quantities has more variability that the mean of independent quantities (remember, $Var(X + Y) = Var(X) + Var(Y) + 2Cov(X, Y)$), and so the more folds the more variability.

In short, increasing the number of folds decreases bias but increases variability. We won't explore these issues further. Instead, we'll just rely on the common practice of using 5 or 10 folds.